# Getting Started to Geomancer

In this demo, we will try to do the following tasks:
- Create a feature representing the distance to the nearest embassy
- Use a different backend from BigQuery to SQLite (Spatialite)
- Generate additional features, bind them into a Spellbook, and share to others

In [1]:
import sys
import warnings
sys.path.append("../")
warnings.filterwarnings('ignore')  # GCP client warns us when we are not using service accounts, let's live on the edge and ignore these warnings

In [2]:
import pandas as pd
from geomancer.spells import DistanceToNearest, NumberOf

In Geomancer, features are represented as spells. We can `cast` is to a set of coordinates. 
Let's first load the points we want to get features for.

In [3]:
df = pd.read_csv("../tests/data/sample_points.csv")
df.head()

,WKT,code
0,POINT (121.0042183 14.6749145),2082
1,POINT (121.0052375 14.6767411),2110
2,POINT (121.009712 14.68067),2082
3,POINT (121.0093311 14.6799482),2082
4,POINT (121.0073296 14.6783498),2082


`WKT` is the column that holds the geometry (in this case, a point with a longitude and latitude), while `code` is just an arbitrary column you don't need to worry about. What geomancer will do is simply add another column pertaining to that particular feature

## Create a feature for the distance to the nearest embassy

Now, let's `cast` the `DistanceToNearest` spell to our DataFrame. It will query from a data warehouse located in BigQuery. You need to configure this in your own organization. If you want setup instructions, please check [this link](https://geomancer.readthedocs.io/en/latest/setup.html#setting-up-your-data-warehouse). We will call this feature `dist_embassy`.

In [4]:
DistanceToNearest("embassy", 
                   source_table="tm-geospatial.ph_osm.gis_osm_pois_free_1",
                   feature_name="dist_embassy").cast(df, dburl="bigquery://tm-geospatial").head()

,WKT,code,dist_embassy
0,POINT (121.0042183 14.6749145),2082,4948.580211
1,POINT (121.0052375 14.6767411),2110,5084.787270
2,POINT (121.009712 14.68067),2082,5319.746371
3,POINT (121.0093311 14.6799482),2082,5256.165257
4,POINT (121.0073296 14.6783498),2082,5162.177598


## Use a different database backend

Earlier, we used BigQuery, but geomancer can also support SQLite (via Spatialite). Let's run the same spell, but on a different database backend

In [5]:
DistanceToNearest("embassy", 
        source_table="gis_osm_pois_free_1",
        feature_name="dist_embassy").cast(df, dburl="sqlite:///data/source.sqlite").head()

,WKT,code,dist_embassy
0,POINT (121.0042183 14.6749145),2082,0.044806
1,POINT (121.0052375 14.6767411),2110,0.045995
2,POINT (121.009712 14.68067),2082,0.047993
3,POINT (121.0093311 14.6799482),2082,0.047431
4,POINT (121.0073296 14.6783498),2082,0.046634


Oh no the output looks different! The output is in meters for BQ, but for SQLite, it's in decimal degrees. So $$\frac{4948.580211 km}{111.32 km} = 0.044806$$
We plan on converting all outputs into meters for consistency (or you can do it by making a PR :p).

## Generate features into a Spellbook

It is also possible to compile features together to share to others. Simply create an instance of a `SpellBook`, and fill it in with a list of Spells you wish to include

In [6]:
from geomancer.spellbook import SpellBook

In [7]:
# Create a spellbook
spellbook = SpellBook(
          spells=[
              DistanceToNearest("embassy",
                                 source_table="ph_osm.gis_osm_pois_free_1",
                                 dburl="bigquery://tm-geospatial",
                                 feature_name="dist_primary"),
              NumberOf("supermarket",
                        source_table="ph_osm.gis_osm_pois_free_1",
                        dburl="bigquery://tm-geospatial",
                        feature_name="num_supermarkets"),
          ])

Using a `SpellBook`, you can do multi-`cast`ing:

In [8]:
spellbook.cast(df).head(5)

,WKT,code,dist_primary,num_supermarkets
0,POINT (121.0042183 14.6749145),2082,4948.580211,181
1,POINT (121.0052375 14.6767411),2110,5084.787270,181
2,POINT (121.009712 14.68067),2082,5319.746371,180
3,POINT (121.0093311 14.6799482),2082,5256.165257,181
4,POINT (121.0073296 14.6783498),2082,5162.177598,181


You can also share your Spellbook to your colleagues, friends, and family!

In [9]:
spellbook.author = "Lj Miranda"
spellbook.description = "Features for Geomancer Demo"
spellbook.to_json("data/features_demo.json")

And you can also read other features from other people and cast it on your own dataset!

In [10]:
some_spellbook = spellbook.read_json("data/some_spellbook.json")
some_spellbook.cast(df).head(5)

,WKT,code,dist_supermarket,num_embassy
0,POINT (121.0042183 14.6749145),2082,0.008973,54
1,POINT (121.0052375 14.6767411),2110,0.008792,54
2,POINT (121.009712 14.68067),2082,0.008990,54
3,POINT (121.0093311 14.6799482),2082,0.008463,54
4,POINT (121.0073296 14.6783498),2082,0.008201,54
